This notebook attempts to solve the Tennis riddler described at:

https://fivethirtyeight.com/features/can-you-figure-out-how-to-beat-roger-federer-at-wimbledon/

To summarize: 
1. you have a 1% chance at winning any given match point against Roger Federer
2. you get to name the starting score

What score should you name to maximize your chance of winning?

The following `Match` class will contain the state of a match to allow for easier game simulation.

In [98]:
from numpy.random import random

class Match:
    '''This class holds the current state of a tennis match.

    Class variables:

    won_match - boolean indicating if you have won the match or None if the match is
      still in progress
    points - an array of the current game points, the first index contains your
      points, the second your opponent's
    games - an array of the current set games, the first index containing your games
      won, the second containing your opponent's
    sets - an array of the current match sets, the first index containing your sets
      won, the second containing your opponent's
    probability_to_win_point - the probability of winning a point against your 
      opponent
    '''
    def __init__(self, points=[0,0], games=[0,0], sets=[0,0], probability_to_win_point=.99):
        self.won_match = None
        self.points = points[:]
        self.games = games[:]
        self.sets = sets[:]
        self.probability_to_win_point = probability_to_win_point
        
    def play_point(self):
        '''Play a point by randomly selecting a number and comparing to the 
           probability of winning a point'''
        if random() >= self.probability_to_win_point:
            self.win_point()
        else:
            self.lose_point()

    def win_point(self):
        '''If the match is still in progress, check for game, set and match wins
           after incrementing your points.'''
        if self.won_match is None:
            self.points[0] += 1
            self.check_points()
    
    def lose_point(self):
        '''If the match is still in progress, check for game, set and match wins
           after incrementing your opponent's points.'''
        if self.won_match is None:    
            self.points[1] += 1
            self.check_points()
    
    def won(self):
        '''Returns if you've won the match'''
        return self.won_match
        
    def check_points(self):
        '''Checks for a game win, set win and match win. If the match has ended,
           won_match is set'''
        game_win = self.check_game()
        if game_win is not None:
            self.games[game_win] += 1
            self.points = [0,0]
        
        set_win = self.check_set()
        if set_win is not None:
            self.sets[set_win] += 1
            self.games = [0,0]
        
        self.check_match()
        
    def check_game(self):
        '''Returns 0 if you have won a game, 1 if your opponent has. A game is 
           won if a player has won at least four points and has a two point 
           advantage.
           
           In the case of a 6-6 set the game is a tie-breaker and a player must
           win at least 7 points and have a two point advantage.'''
        if self.games == [6,6]:
            if self.points[0] >= 7 and self.points[0] - self.points[1] >= 2:
                return 0
            elif self.points[1] >= 7 and self.points[1] - self.points[0] >= 2:
                return 1
        elif self.points[0] >= 4 and self.points[0] - self.points[1] >= 2:
            return 0
        elif self.points[1] >= 4 and self.points[1] - self.points[0] >= 2:
            return 1
    
    def check_set(self):
        '''Returns 0 if you have won a set, 1 if your opponent has. A set is 
           won if a player has won at least six games and has a two game 
           advantage.
           
           In the case of a 6-6 set the game is a tie-breaker, and the winner
           takes the set point.'''
        if self.games == [7,6]:
            return 0
        elif self.games == [6,7]:
            return 1
        elif self.games[0] >= 6 and self.games[0] - self.games[1] >= 2:
            return 0
        elif self.games[1] >= 6 and self.games[1] - self.games[0] >= 2:
            return 1
    
    def check_match(self):
        '''Checks if the match has ended and sets won_match to True if you've won,
        False if your opponent has.'''
        if self.sets[0] == 3: 
            self.won_match = True
        elif self.sets[1] == 3:
            self.won_match = False
    
    def __str__(self):
        match_str = ""
        if self.won_match is not None:
            match_str = "won!" if self.won_match else "lost!"
        return "game: {}-{}\nset: {}-{}\nmatch: {}-{}\n{}".format(self.points[0], self.points[1],
                                                                  self.games[0], self.games[1], self.sets[0],
                                                                  self.sets[1], match_str)
    
    def __repr__(self):
        return str(self)
    

In [133]:
# Define a method to simulate a number of games, and average them to get the
# percentage of games won
def run_simulation(num_runs, start_points, start_games, start_sets):
    wins = 0
    for ii in range(0, num_runs):
        current_match = Match(start_points, start_games, start_sets, probability)
        while current_match.won() is None:
            current_match.play_point()
        wins += 1 if current_match.won() else 0
            
    print("Percentage of games won:", wins/num_runs)

In [137]:
# Run the simulation 10,000 times and average to get an estimate of 
# the percentage of games won at these starting points
num_runs = 10000

# At first glance it would seem like 2-0 sets, 5-0 games, and 40-love
# would maximize your chances of winning. Let's see what that looks like:
run_simulation(num_runs, [3,0], [5,0], [2,0])

Percentage of games won: 0.0307


In [138]:
# It turns out, however, that we can do better by entering into a tie-breaker game:
run_simulation(num_runs, [6,0], [6,6], [2,0])

Percentage of games won: 0.0563
